# Answering Business Questions for a Music Record Store using SQL

The aim of the project is to explore a database using SQL and answer some business questions

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%bash
ls

Basics.ipynb
chinook.db
chinook-unmodified.db


In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
%%sql
SELECT *
FROM sqlite_master
WHERE type IN ("table","view");

Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


--------
Selecting Albums to Purchase
----------

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

- Regal:	Hip-Hop
- Red Tone:	Punk
- Meteor and the Girls:	Pop
- Slim Jim Bites:	Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

Finding out which genres sell the most tracks in the USA:

In [5]:
%%sql
WITH track_sold AS (
    SELECT
        SUM(il.quantity) AS total_sold,
        g.name AS genre
    FROM track AS t
    LEFT JOIN genre AS g ON g.genre_id = t.genre_id
    LEFT JOIN invoice_line AS il ON il.track_id = t.track_id
    LEFT JOIN invoice AS i ON i.invoice_id = il.invoice_id
    WHERE i.billing_country == 'USA'
    GROUP BY 2 )

SELECT 
    genre,
    total_sold,
    ROUND(CAST(total_sold AS FLOAT) / (SELECT SUM(total_sold) 
                                       FROM track_sold) * 100,1)
                                        || " %" AS total_sold_pct
FROM track_sold
ORDER BY 2 DESC ;

Done.


genre,total_sold,total_sold_pct
Rock,561,53.4 %
Alternative & Punk,130,12.4 %
Metal,124,11.8 %
R&B/Soul,53,5.0 %
Blues,36,3.4 %
Alternative,35,3.3 %
Latin,22,2.1 %
Pop,22,2.1 %
Hip Hop/Rap,20,1.9 %
Jazz,14,1.3 %


Following genres are most sold in hte USA:
- Rock
- Alternative & Punk	
- Metal

Based on the ranking of the genres it is better to purchase the following 3 albums:
- Red Tone: Punk
- Slim Jim Bites	Blues
- Meteor and the Girls	Pop

---------
Analyzing Employee Sales Performance
-------------

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. 
We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.



In [6]:
%%sql
SELECT
    e.employee_id AS employee_id,
    e.first_name || " " || e.last_name AS name,
    ROUND(SUM(i.total)) AS total_amount,
    substr(e.hire_date, 1, instr(e.hire_date,' ')) AS hire_date,
    e.title AS title
FROM employee AS e
INNER JOIN customer AS c ON c.support_rep_id = e.employee_id
INNER JOIN INVOICE AS i ON i.customer_id = c.customer_id
GROUP BY 1 
ORDER BY 3 DESC ;

Done.


employee_id,name,total_amount,hire_date,title
3,Jane Peacock,1732.0,2017-04-01,Sales Support Agent
4,Margaret Park,1584.0,2017-05-03,Sales Support Agent
5,Steve Johnson,1394.0,2017-10-17,Sales Support Agent


Overall Jane has the highest toal amount of sales. 
Hoewever, it seems that the difference in total sales are more related to the hiring date of the sales supoort agents

-----------
Analyzing Sales by Country
-----------

Next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, we  group these customers as "Other" in your analysis. And force the ordering of "Other" to be the last in the analysis.

In [7]:
%%sql
WITH sales_country AS (
    SELECT 
        CASE 
            WHEN (SELECT COUNT(DISTINCT customer_id) 
                  FROM customer
                  WHERE country = c.country) = 1 THEN 'Other'
            ELSE c.country
        END AS country,
        COUNT(DISTINCT c.customer_id) AS total_customer,
        ROUND(SUM(i.total)) AS total_sales,
        COUNT(distinct i.invoice_id) AS total_orders
    FROM customer AS c
    INNER JOIN invoice AS i ON i.customer_id = c.customer_id
    GROUP BY 1
    ORDER BY 3 DESC)

SELECT
    country,
    total_customer,
    total_sales,
    ROUND(total_sales / total_customer,2) AS avg_sales_customer,
    ROUND(total_sales / total_orders,2) AS avg_sales_order
FROM (
    SELECT
        sc.*,
        CASE
            WHEN sc.country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM sales_country AS sc)
ORDER BY sort ASC, 3 DESC ;


Done.


country,total_customer,total_sales,avg_sales_customer,avg_sales_order
USA,13,1040.0,80.0,7.94
Canada,8,536.0,67.0,7.05
Brazil,5,428.0,85.6,7.02
France,5,389.0,77.8,7.78
Germany,4,335.0,83.75,8.17
Czech Republic,2,273.0,136.5,9.1
United Kingdom,3,246.0,82.0,8.79
Portugal,2,185.0,92.5,6.38
India,2,183.0,91.5,8.71
Other,15,1095.0,73.0,7.45


------
Albulm vs Individual Tracks
----------

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.



Categorizing each invoice as either an album purchase or not, and calculating the following summary statistics:
- Number of invoices
- Percentage of invoices

In [41]:
%%sql
WITH all_track AS (
    SELECT 
        i.invoice_id,
        SUM(CASE 
              WHEN (SELECT il1.track_id 
                  FROM invoice AS i1
                  INNER JOIN invoice_line AS il1 ON 
                        il1.invoice_id = i1.invoice_id
                  WHERE i1.invoice_id = i.invoice_id
                    
                  EXCEPT
                    
                  SELECT t2.track_id 
                  FROM album AS a2
                  INNER JOIN track AS t2 ON t2.album_id = a2.album_id 
                  WHERE a2.album_id = a.album_id) IS NULL
            
                  AND
            
                  (SELECT t2.track_id 
                  FROM album AS a2
                  INNER JOIN track AS t2 ON t2.album_id = a2.album_id 
                  WHERE a2.album_id = a.album_id
                   
                  EXCEPT
                   
                  SELECT il1.track_id 
                  FROM invoice AS i1
                  INNER JOIN invoice_line AS il1 ON 
                       il1.invoice_id = i1.invoice_id
                  WHERE i1.invoice_id = i.invoice_id) IS NULL 
                THEN 0
                ELSE 1
        END) AS if_all_tracks
    FROM invoice AS i
    INNER JOIN invoice_line AS il ON il.invoice_id = i.invoice_id
    INNER JOIN track AS t ON t.track_id = il.track_id
    INNER JOIN album AS a ON a.album_id = t.album_id
    GROUP BY 1 )
    
SELECT 
    CASE
        WHEN (SELECT if_all_tracks 
              FROM all_track
              WHERE invoice_id = at.invoice_id) = 0 THEN 'Yes'
        ELSE 'No'
    END AS full_album,
    COUNT(DISTINCT at.invoice_id) AS number_of_invoices,
    ROUND(CAST(COUNT(DISTINCT at.invoice_id) AS FLOAT) / 
          (SELECT COUNT(*) FROM all_track) * 100,1) AS pct_of_invoices
FROM all_track AS at
GROUP BY full_album ;

Done.


full_album,number_of_invoices,pct_of_invoices
No,500,81.4
Yes,114,18.6


About 18% of all purchases are "full-album" purchases, therefore changing the strategy might result on a loss of about 18%.

-------
Which artist is used in the most playlists?
-----------

In [56]:
%%sql
SELECT 
    ar.name AS artist,
    COUNT(DISTINCT pt.playlist_id) AS number_of_playlist
FROM playlist_track AS pt
INNER JOIN track AS t ON t.track_id = pt.track_id
INNER JOIN album AS al ON al.album_id = t.album_id
INNER JOIN artist AS ar ON ar.artist_id = al.artist_id

GROUP BY 1
ORDER BY 2 DESC
LIMIT 10 ;

Done.


artist,number_of_playlist
Eugene Ormandy,7
Academy of St. Martin in the Fields & Sir Neville Marriner,6
Berliner Philharmoniker & Herbert Von Karajan,6
English Concert & Trevor Pinnock,6
The King's Singers,6
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",5
Adrian Leaper & Doreen de Feis,5
Alberto Turco & Nova Schola Gregoriana,5
Antal Doráti & London Symphony Orchestra,5
Barry Wordsworth & BBC Concert Orchestra,5


----------
How many tracks have been purchased vs not purchased?
------------

In [55]:
%%sql
SELECT 
    CASE
        WHEN(
            il.track_id = t.track_id
        ) 
        THEN "Yes"
        ELSE "No"
    END AS purchased,
    COUNT(DISTINCT t.track_id) AS number_of_tracks,
    ROUND(CAST(COUNT(DISTINCT t.track_id) AS FLOAT) / 
          (SELECT COUNT(*) FROM track) * 100,1) AS pct_of_all
    
FROM track AS t
LEFT JOIN invoice_line AS il ON il.track_id = t.track_id

GROUP BY 1 ;

Done.


purchased,number_of_tracks,pct_of_all
No,1697,48.4
Yes,1806,51.6


-------
Do protected vs non-protected media types have an effect on popularity?
-----------

In [59]:
%%sql
SELECT * FROM media_type

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


In [86]:
%%sql
WITH 
    purchase_protected AS (SELECT 
        t.track_id AS track_id,
        t.name AS track_name,
        SUM(il.quantity) AS number_of_purchase,
        mt.name AS media_type,
        CASE
            WHEN(
                mt.name LIKE "Protected%"
            )
            THEN 'Yes'
            ELSE 'No'
        END AS protected
    FROM track AS t
    LEFT JOIN invoice_line AS il ON il.track_id = t.track_id
    LEFT JOIN media_type AS mt ON mt.media_type_id = t.media_type_id

    GROUP BY 1
    ORDER BY 2 DESC)

SELECT 
    protected,
    SUM(number_of_purchase) AS number_of_purchased,
    COUNT(DISTINCT track_id) AS number_of_tracks,
    ROUND(CAST(SUM(number_of_purchase) AS FLOAT) / 
          COUNT(DISTINCT track_id),2) AS number_of_purchase_per_track
    
FROM purchase_protected
GROUP BY 1 ;

    

Done.


protected,number_of_purchased,number_of_tracks,number_of_purchase_per_track
No,4315,3052,1.41
Yes,442,451,0.98


Non protected tracks have higher avegrage purchse per track therfore one can conclude that they are more poular